In [ ]:
!pip install python-gdcm pylibjpeg --no-index --find-links "../input/rsna-wheels/"

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch

device = torch.device('cpu')

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
import torch.nn as nn
from torchvision import transforms
import torch.optim as optim
import numpy as np
import os
import pydicom
from pydicom.pixel_data_handlers import apply_windowing
import torch.nn.functional as F
from torchmetrics.classification import BinaryF1Score

In [ ]:
roi_extractor_model = torch.hub.load('/kaggle/input/yolov5-repo', 'custom', path='/kaggle/input/rsna-breast-cancer-detection-roi-model/rsna-roi-003.pt', source='local')

### 

In [ ]:
# helper functions
def read_dicom_with_windowing(dcm_file):
    # from: https://www.kaggle.com/code/davidbroberts/mammography-apply-windowing/
    im = pydicom.dcmread(dcm_file)
    data = im.pixel_array
    
    # This line is the only difference in the two functions
    data = apply_windowing(data, im)
    
    if im.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    else:
        data = data - np.min(data)
        
    if np.max(data) != 0:
        data = data / np.max(data)
    data=(data * 255).astype(np.uint8)

    return data

In [ ]:
def crop_yolo_ROI(image):
    # https://www.kaggle.com/code/remekkinas/breast-cancer-roi-brest-extractor/notebook
    detections = roi_extractor_model(image)
    
    bbox_df = detections.pandas().xyxy[0]
    bbox_df.drop(["name"], axis=1, inplace=True)
    bboxes = bbox_df.astype(int).to_dict(orient="records")
    
    if len(bboxes) == 0:
        return image
    
    if len(bboxes) != 1:
        print(bboxes)
        print("More than one ROI detected")

    bbox = bboxes[0]
    
    image = image[bbox["ymin"]:bbox["ymax"], bbox["xmin"]:bbox["xmax"]]
    return image

In [ ]:
class BreastCancerDataset(torch.utils.data.Dataset):
    # image to cancer
    def __init__(self, df, dataset_folder, transform=None):
        self.dataset_folder = dataset_folder
        self.transform = transform
        
        if "breast_id" not in df.columns:
            df["breast_id"] = df["patient_id"].astype(str) + "_" + df["laterality"]

        if "image_path" not in df.columns:
            df["image_path"] = df["patient_id"].astype(str) + "/" + df["image_id"].astype(str) + ".dcm"

        self.df = df
        if 'cancer' in self.df.columns:
            self.subset = 'train'
        else:
            self.subset = 'test'

    def __len__(self):
        return len(self.df.groupby("breast_id"))

    def __getitem__(self, idx):
        # return MLO and CC images from given breast
        breast_id = self.df["breast_id"].unique()[idx]
        breast_df = self.df[self.df["breast_id"] == breast_id]
        
        MLO = breast_df[breast_df["view"] == "MLO"]
        CC = breast_df[breast_df["view"] == "CC"]

        MLO_image_path = os.path.join(self.dataset_folder, self.subset+ "_images", MLO["image_path"].values[0])
        CC_image_path = os.path.join(self.dataset_folder, self.subset+ "_images", CC["image_path"].values[0])

        MLO_image = read_dicom_with_windowing(MLO_image_path)
        CC_image = read_dicom_with_windowing(CC_image_path)
        
        MLO_image = crop_yolo_ROI(MLO_image)
        CC_image = crop_yolo_ROI(CC_image)

        MLO_image = torch.from_numpy(MLO_image)
        CC_image = torch.from_numpy(CC_image)
        
        if self.transform:  # normalization and augmentation are in here
            MLO_image = self.transform(MLO_image)
            CC_image = self.transform(CC_image)
        
        # add these images into a 2 channel image
        image = torch.cat([MLO_image, CC_image], dim=0)

        if self.subset == 'train':
            label = breast_df['cancer'].values[0]
            return image, label
        
        
        return image

In [ ]:
train_df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
test_df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')

dataset_folder = '/kaggle/input/rsna-breast-cancer-detection'

transform = transforms.Compose([ # also add augmentation
    transforms.ToPILImage(),
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
])

train_dataset = BreastCancerDataset(train_df, dataset_folder, transform=transform)
test_dataset = BreastCancerDataset(test_df, dataset_folder, transform=transform)

In [ ]:
print(test_df)
print(train_df.columns)

In [ ]:
# plot first batch
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 4, shuffle=False)
images, labels = next(iter(train_loader))

print(images.shape, labels.shape)

In [ ]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        input_size = 100*100*.25*.25*12
        self.conv1 = nn.Conv2d(2, 6, 5, padding='same')
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 12, 5, padding='same')
        self.fc1 = nn.Linear(int(input_size), int(input_size * .33))
        #self.fc2 = nn.Linear(int(input_size*.33), int(input_size * .33**2))
        self.fc3 = nn.Linear(int(input_size * .33), 1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc3(x))
        x = torch.flatten(x)
        return x

In [ ]:
cnn_model = CNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.SGD(cnn_model.parameters(), lr=.001, momentum=.9)

In [ ]:
num_epochs = 1
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    f1 = 0
    for idx, (image, target) in enumerate(train_loader):
        target = target.to(device)
        image = image.to(device)
        outputs=cnn_model(image)

        loss = criterion(outputs, target.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        output_binary = torch.Tensor.cpu(outputs).detach().numpy()
        output_binary[output_binary > .5] = 1
        output_binary[output_binary<=.5] = 0
        f1 += f1_score(output_binary, torch.Tensor.cpu(target).numpy())
        if idx%5 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], {idx+1}/{n_total_steps}, Loss: {loss.item():.4f}, F1: {f1/(5):.4f}')
            f1 = 0
        if idx > 50:
            break

In [ ]:
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = cnn_model(images)
outputs = torch.Tensor.cpu(outputs).numpy()
print(outputs)
submission = {}
submission['prediction_id'] = ['10008_L', '10008_R']
submission['cancer'] = outputs
submission_df = pd.DataFrame(submission)
print(submission_df)
submission_df.to_csv('submission.csv', index=False)
